In [4]:
!pip install beautifulsoup4 webdriver_manager


In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,738 kB]
Get:13 https://developer.download.nvidia.com/compu

In [2]:
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver
!pip install chromedriver-autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import sys
import os
import time
import requests
import pandas as pd
from tqdm import tqdm

import chromedriver_autoinstaller  # setup chrome options
from bs4 import BeautifulSoup

import urllib.request
from urllib.request import urlretrieve

In [38]:
chrome_path = "/usr/lib/chromium-browser/chromedriver"

In [39]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

# 크롬 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

In [42]:

# 페이지 열기
url = "https://unipass.customs.go.kr/clip/index.do"
driver.get(url)

# JavaScript가 실행될 시간을 기다리기
driver.implicitly_wait(10)  # 최대 10초 대기

# 페이지의 HTML 가져오기
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# 세계HS 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.CLASS_NAME,"M_ULS0200000000")
click_box.click()

# HS해설서 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="LEFTMENU_LNK_UI-ULS-0202-001Q"]/span')
click_box.click()

# 조회 창을 찾아 click_box 변수에 저장 (By.XPATH 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="searchVo"]/div[2]/footer/button/span')
driver.execute_script("arguments[0].click();", click_box)


# Wait for the table to load
time.sleep(1)  # Adjust the wait time as needed

# Locate the table
table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')

# Parse the table HTML using BeautifulSoup
soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')

# Extract table rows
rows = soup.find_all('tr')

# Initialize an empty DataFrame
data = pd.DataFrame(columns=["구분", "년도", "단위", "HS분류", "내용", "Left Area", "Right Area"])

# Loop through all 135 pages
for page in tqdm(range(1, 135)):
    print(f"Processing page {page}...")

    # Locate the table
    table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')

    # Parse the table HTML using BeautifulSoup
    soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')

    # Extract table rows
    rows = soup.find_all('tr')

    # Iterate through the rows and extract data
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
        if cols:  # Check if the row has data
            # <td class="ellipsis" title="류"> 요소 찾기
            dtl_link = row.find('a', class_='dtlInfo')

            # Click on the `dtlInfo` link
            # dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)

            dtl_element=driver.find_element(By.XPATH, f"//a[contains(text(), '{dtl_link.text[:25]}')]")

            driver.execute_script("arguments[0].click();", dtl_element)  # Use JavaScript click to avoid staleness
            time.sleep(0.8)  # Wait for the leftArea and rightArea to load

            # Extract text from leftArea
            left_area = driver.find_element(By.ID, 'divLft_tab4')
            left_area_text = left_area.text.strip()

            # Extract text from rightArea
            right_area = driver.find_element(By.ID, 'divRght_tab4')
            right_area_text = right_area.text.strip()

            # Add data to the DataFrame
            data.loc[len(data)] = cols + [left_area_text, right_area_text]

    # Navigate to the next page
    try:
        wait=WebDriverWait(driver, 10)
        next_page = wait.until(EC.element_to_be_clickable((By.XPATH, f'//div[@class="paging"]//a[@href="#{page + 1}"]')))
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(2)  # Wait for the page to load
    except Exception as e:
        print(f"Could not navigate to page {page + 1}: {e}")
        break

# Display the DataFrame

  0%|          | 0/135 [00:00<?, ?it/s]

Processing page 1...
류 설명
류 설명


  1%|          | 1/135 [00:11<24:38, 11.03s/it]

Processing page 2...


  1%|▏         | 2/135 [00:22<24:24, 11.01s/it]

...

Processing page 134...
류 설명


 99%|█████████▉| 134/135 [25:31<00:11, 11.38s/it]

Processing page 135...


 99%|█████████▉| 134/135 [25:47<00:11, 11.55s/it]

Could not navigate to page 136: Message: 
Stacktrace:
#0 0x581b3023c34a <unknown>
#1 0x581b2fd526e0 <unknown>
#2 0x581b2fda13e6 <unknown>
#3 0x581b2fda1681 <unknown>
#4 0x581b2fde6b04 <unknown>
#5 0x581b2fdc548d <unknown>
#6 0x581b2fde3ed7 <unknown>
#7 0x581b2fdc5203 <unknown>
#8 0x581b2fd93cc0 <unknown>
#9 0x581b2fd94c9e <unknown>
#10 0x581b30209d3b <unknown>
#11 0x581b3020dcc2 <unknown>
#12 0x581b301f6b6c <unknown>
#13 0x581b3020e837 <unknown>
#14 0x581b301dc10f <unknown>
#15 0x581b3022b5a8 <unknown>
#16 0x581b3022b770 <unknown>
#17 0x581b3023b1c6 <unknown>
#18 0x7c6d3b72dac3 <unknown>



In [43]:

# Save the data to a CSV file
data.to_csv('HSdescription_data.csv', index=False, encoding='utf-8-sig')

In [44]:
data

,구분,년도,단위,HS분류,내용,Left Area,Right Area
0,WCO,2022,류,01,제1류 살아 있는 동물 주: 1. 이 류에는 다음 각 목의 것을 제외한 모든 살아 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
1,WCO,2022,부,01,제 1 부 살아 있는 동물과 동물성 생산품 주: 1. 이 부에 열거된 동물의 특정 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
2,WCO,2022,호,0101,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+) - 말 0101.21 -- 번식...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
3,WCO,2022,호,0102,01.02 - 살아 있는 소(+) - 축우(畜牛) 0102.21 -- 번식용 010...,01.02 - 살아 있는 소(+)\n- 축우(畜牛)\n0102.21 - 번식용\n0...,01.02 Live bovine animals (+).\n- Cattle :\n01...
4,WCO,2022,호,0103,01.03 - 살아 있는 돼지(+) 0103.10 - 번식용 - 기타 0103.91...,01.03 - 살아 있는 돼지(+)\n0103.10 - 번식용\n- 기타\n0103...,01.03 - Live swine (+).\n0103.10 - Pure bred b...
...,...,...,...,...,...,...,...
1341,WCO,2022,호,9702,97.02 - 오리지널 동판화ㆍ목판화ㆍ석판화 9702.10 - 제작 후 100년을 ...,97.02 - 오리지널 동판화ㆍ목판화ㆍ석판화\n9702.10 - 제작 후 100년을...,"97.02 Original engravings, prints and lithogra..."
1342,WCO,2022,호,9703,97.03 - 오리지널 조각과 조상(彫像)(어떤 재료라도 가능하다) 9703.10 ...,97.03 - 오리지널 조각과 조상(彫像)(어떤 재료라도 가능하다)\n9703.10...,"97.03 Original sculptures and statuary, in any..."
1343,WCO,2022,호,9704,97.04 - 우표ㆍ수입인지ㆍ우편요금 별납증서ㆍ초일(初日)봉투ㆍ우편엽서류와 이와 유...,97.04 - 우표ㆍ수입인지ㆍ우편요금 별납증서ㆍ초일(初日)봉투ㆍ우편엽서류와 이와 유...,"97.04 Postage or revenue stamps, stamp-postmar..."
1344,WCO,2022,호,9705,97.05 - 수집품과 표본[고고학ㆍ민족학ㆍ사학ㆍ동물학ㆍ식물학ㆍ광물학ㆍ해부학ㆍ고생물...,97.05 - 수집품과 표본[고고학ㆍ민족학ㆍ사학ㆍ동물학ㆍ식물학ㆍ광물학ㆍ해부학ㆍ고생물...,97.05 Collections and collectors’ pieces of ar...


In [45]:
data.tail()

,구분,년도,단위,HS분류,내용,Left Area,Right Area
1341,WCO,2022,호,9702,97.02 - 오리지널 동판화ㆍ목판화ㆍ석판화 9702.10 - 제작 후 100년을 ...,97.02 - 오리지널 동판화ㆍ목판화ㆍ석판화\n9702.10 - 제작 후 100년을...,"97.02 Original engravings, prints and lithogra..."
1342,WCO,2022,호,9703,97.03 - 오리지널 조각과 조상(彫像)(어떤 재료라도 가능하다) 9703.10 ...,97.03 - 오리지널 조각과 조상(彫像)(어떤 재료라도 가능하다)\n9703.10...,"97.03 Original sculptures and statuary, in any..."
1343,WCO,2022,호,9704,97.04 - 우표ㆍ수입인지ㆍ우편요금 별납증서ㆍ초일(初日)봉투ㆍ우편엽서류와 이와 유...,97.04 - 우표ㆍ수입인지ㆍ우편요금 별납증서ㆍ초일(初日)봉투ㆍ우편엽서류와 이와 유...,"97.04 Postage or revenue stamps, stamp-postmar..."
1344,WCO,2022,호,9705,97.05 - 수집품과 표본[고고학ㆍ민족학ㆍ사학ㆍ동물학ㆍ식물학ㆍ광물학ㆍ해부학ㆍ고생물...,97.05 - 수집품과 표본[고고학ㆍ민족학ㆍ사학ㆍ동물학ㆍ식물학ㆍ광물학ㆍ해부학ㆍ고생물...,97.05 Collections and collectors’ pieces of ar...
1345,WCO,2022,호,9706,97.06 - 골동품(제작 후 100년을 초과한 것으로 한정한다) 9706.10 -...,97.06 - 골동품(제작 후 100년을 초과한 것으로 한정한다)\n9706.10 ...,97.06 Antiques of an age exceeding 100 years.\...
